# Scheduling, Preemption and Eviction

# Scheduling

### Kubernetes Scheduler

### Scheduling overview

### kube-scheduler

### Node selection in kube-scheduler 

### 範例

In [ ]:
kubectl run web --image=nginx
kubectl describe pod web

<img src='./img/29.png'>

# Assigning Pods to Nodes

### Node labels

### Node isolation/restriction

### nodeSelector

### 範例: Assign Pods to Nodes

##### Before you begin

##### Add a label to a node

In [ ]:
kubectl get nodes --show-labels

In [ ]:
kubectl label nodes <your-node-name> disktype=ssd

In [ ]:
kubectl get nodes --show-labels

##### 🔥 Create a pod that gets scheduled to your chosen node 🔥

In [ ]:
apiVersion: v1
kind: Pod
metadata:
  name: nginx
  labels:
    env: test
spec:
  containers:
  - name: nginx
    image: nginx
    imagePullPolicy: IfNotPresent
  nodeSelector:
    disktype: ssd

In [ ]:
kubectl apply -f https://k8s.io/examples/pods/pod-nginx.yaml

In [ ]:
kubectl get pods --output=wide

##### 🔥 Create a pod that gets scheduled to specific node 🔥

In [ ]:
apiVersion: v1
kind: Pod
metadata:
  name: nginx
spec:
  nodeName: foo-node # schedule pod to specific node
  containers:
  - name: nginx
    image: nginx
    imagePullPolicy: IfNotPresent

# Affinity and anti-affinity

### Node affinity

In [ ]:
apiVersion: v1
kind: Pod
metadata:
  name: with-node-affinity
spec:
  affinity:
    nodeAffinity:
      requiredDuringSchedulingIgnoredDuringExecution:
        nodeSelectorTerms:
        - matchExpressions:
          - key: topology.kubernetes.io/zone
            operator: In
            values:
            - antarctica-east1
            - antarctica-west1
      preferredDuringSchedulingIgnoredDuringExecution:
      - weight: 1
        preference:
          matchExpressions:
          - key: another-node-label-key
            operator: In
            values:
            - another-node-label-value
  containers:
  - name: with-node-affinity
    image: registry.k8s.io/pause:2.0

### 範例

##### Add a label to a node

In [ ]:
kubectl get nodes --show-labels

In [ ]:
NAME      STATUS    ROLES    AGE     VERSION        LABELS
worker0   Ready     <none>   1d      v1.13.0        ...,kubernetes.io/hostname=worker0
worker1   Ready     <none>   1d      v1.13.0        ...,kubernetes.io/hostname=worker1
worker2   Ready     <none>   1d      v1.13.0        ...,kubernetes.io/hostname=worker2

In [ ]:
kubectl label nodes <your-node-name> disktype=ssd

In [ ]:
kubectl get nodes --show-labels

##### Schedule a Pod using required node affinity

In [ ]:
apiVersion: v1
kind: Pod
metadata:
  name: nginx
spec:
  affinity:
    nodeAffinity:
      requiredDuringSchedulingIgnoredDuringExecution:
        nodeSelectorTerms:
        - matchExpressions:
          - key: disktype
            operator: In
            values:
            - ssd            
  containers:
  - name: nginx
    image: nginx
    imagePullPolicy: IfNotPresent

In [ ]:
kubectl apply -f https://k8s.io/examples/pods/pod-nginx-required-affinity.yaml

In [ ]:
kubectl get pods --output=wide

##### Schedule a Pod using preferred node affinity

In [ ]:
apiVersion: v1
kind: Pod
metadata:
  name: nginx
spec:
  affinity:
    nodeAffinity:
      preferredDuringSchedulingIgnoredDuringExecution:
      - weight: 1
        preference:
          matchExpressions:
          - key: disktype
            operator: In
            values:
            - ssd          
  containers:
  - name: nginx
    image: nginx
    imagePullPolicy: IfNotPresent

In [ ]:
kubectl apply -f https://k8s.io/examples/pods/pod-nginx-preferred-affinity.yaml

In [ ]:
kubectl get pods --output=wide

### Node affinity weight 

In [ ]:
apiVersion: v1
kind: Pod
metadata:
  name: with-affinity-anti-affinity
spec:
  affinity:
    nodeAffinity:
      requiredDuringSchedulingIgnoredDuringExecution:
        nodeSelectorTerms:
        - matchExpressions:
          - key: kubernetes.io/os
            operator: In
            values:
            - linux
      preferredDuringSchedulingIgnoredDuringExecution:
      - weight: 1
        preference:
          matchExpressions:
          - key: label-1
            operator: In
            values:
            - key-1
      - weight: 50
        preference:
          matchExpressions:
          - key: label-2
            operator: In
            values:
            - key-2
  containers:
  - name: with-node-affinity
    image: registry.k8s.io/pause:2.0

### Node affinity per scheduling profile

In [ ]:
apiVersion: kubescheduler.config.k8s.io/v1beta3
kind: KubeSchedulerConfiguration

profiles:
  - schedulerName: default-scheduler
  - schedulerName: foo-scheduler
    pluginConfig:
      - name: NodeAffinity
        args:
          addedAffinity:
            requiredDuringSchedulingIgnoredDuringExecution:
              nodeSelectorTerms:
              - matchExpressions:
                - key: scheduler-profile
                  operator: In
                  values:
                  - foo

### Inter-pod affinity and anti-affinity

### Types of inter-pod affinity and anti-affinity

### 範例

In [ ]:
apiVersion: v1
kind: Pod
metadata:
  name: with-pod-affinity
spec:
  affinity:
    podAffinity:
      requiredDuringSchedulingIgnoredDuringExecution:
      - labelSelector:
          matchExpressions:
          - key: security
            operator: In
            values:
            - S1
        topologyKey: topology.kubernetes.io/zone
    podAntiAffinity:
      preferredDuringSchedulingIgnoredDuringExecution:
      - weight: 100
        podAffinityTerm:
          labelSelector:
            matchExpressions:
            - key: security
              operator: In
              values:
              - S2
          topologyKey: topology.kubernetes.io/zone
  containers:
  - name: with-pod-affinity
    image: registry.k8s.io/pause:2.0

### nodeName

In [ ]:
apiVersion: v1
kind: Pod
metadata:
  name: nginx
spec:
  containers:
  - name: nginx
    image: nginx
  nodeName: kube-01

### Pod topology spread constraints

### Taints and Tolerations

##### Concepts

In [ ]:
kubectl taint nodes node1 key1=value1:NoSchedule

In [ ]:
kubectl taint nodes node1 key1=value1:NoSchedule-

In [ ]:
tolerations:
- key: "key1"
  operator: "Equal"
  value: "value1"
  effect: "NoSchedule"

In [ ]:
tolerations:
- key: "key1"
  operator: "Exists"
  effect: "NoSchedule"

In [ ]:
apiVersion: v1
kind: Pod
metadata:
  name: nginx
  labels:
    env: test
spec:
  containers:
  - name: nginx
    image: nginx
    imagePullPolicy: IfNotPresent
  tolerations:
  - key: "example-key"
    operator: "Exists"
    effect: "NoSchedule"`

In [ ]:
kubectl taint nodes node1 key1=value1:NoSchedule
kubectl taint nodes node1 key1=value1:NoExecute
kubectl taint nodes node1 key2=value2:NoSchedule

In [ ]:
tolerations:
- key: "key1"
  operator: "Equal"
  value: "value1"
  effect: "NoSchedule"
- key: "key1"
  operator: "Equal"
  value: "value1"
  effect: "NoExecute"

In [ ]:
tolerations:
- key: "key1"
  operator: "Equal"
  value: "value1"
  effect: "NoExecute"
  tolerationSeconds: 3600

# Cordoning

In [ ]:
kubectl cordon <node_name>

In [ ]:
kubectl uncordon <node_name>

# drain

In [ ]:
kubectl drain <node name> --ingore-daemonsets

### 範例: Safely Drain a Node

##### Before you begin

##### (Optional) Configure a disruption budget

##### Use kubectl drain to remove a node from service

In [ ]:
kubectl get nodes

In [ ]:
kubectl drain <node name>

In [ ]:
kubectl uncordon <node name>

##### Draining multiple nodes in parallel

##### The Eviction API